<a href="https://colab.research.google.com/github/Witcape/Emotion_Recognition_in_Live_Video_Data/blob/main/Emotion_Recognition_in_Video_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### STEP 1 - Import all the Libraries and google drive

In [2]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import sklearn
import tensorflow as tf
import pandas as pd
import sklearn
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.activations import linear,relu,sigmoid
import cv2
import os
import PIL
import pathlib
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
loss = history.history['loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: ignored

## Step 2 - Loading Data & Preprocessing

In [4]:
train_data_dir = '/content/gdrive/MyDrive/Human emotions/train'
test_data_dir = '/content/gdrive/MyDrive/Human emotions/test'

In [4]:
train_data_dir_pil = pathlib.Path('/content/gdrive/MyDrive/Human emotions/train')
test_data_dir_pil = pathlib.Path('/content/gdrive/MyDrive/Human emotions/test')

In [ ]:
flower_imgage_dict = {
    'angry' : list(train_data_dir_pil.glob('angry/*')),
    'disgusted' : list(train_data_dir_pil.glob('disgusted/*')),
    'fearful' : list(train_data_dir_pil.glob('fearful/*')),
    'happy' : list(train_data_dir_pil.glob('happy/*')),
    'neutral' : list(train_data_dir_pil.glob('neutral/*')),
    'sad' : list(train_data_dir_pil.glob('sad/*')),
    'surprised' : list(train_data_dir_pil.glob('surprised/*')),
}

flower_label_dict = {
    'angry' : 0,
    'disgusted' : 1,
    'fearful' : 2,
    'happy' : 3,
    'neutral' : 4,
    'sad' : 5,
    'surprised' :6
}

In [5]:
picture_size = 48
batch_size = 64
no_of_classes = 7
epochs = 48

datagen_train = ImageDataGenerator (
                                    rescale=1./255,
                                    validation_split=0.2
                                    )

datagen_val   = ImageDataGenerator (
                                    rescale=1./255
                                    )

In [6]:
train_set = datagen_train.flow_from_directory(
    train_data_dir,
    target_size=(picture_size, picture_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_set = datagen_val.flow_from_directory(
    test_data_dir,
    target_size=(picture_size, picture_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 29031 images belonging to 7 classes.
Found 7378 images belonging to 7 classes.


In [7]:
train_data = tf.keras.utils.image_dataset_from_directory(train_data_dir)
test_data = tf.keras.utils.image_dataset_from_directory(test_data_dir)

Found 29031 files belonging to 7 classes.
Found 7378 files belonging to 7 classes.


In [ ]:
train_data_iterator = train_data.as_numpy_iterator()
test_data_iterator = test_data.as_numpy_iterator()

In [ ]:
train_batch = train_data_iterator.next()
test_batch = test_data_iterator.next()

In [ ]:
len(test_batch)

2

In [ ]:
train_batch[0].shape

(32, 256, 256, 3)

In [ ]:
train_data = train_data.map(lambda x,y: (x/255,y))
test_data = test_data.map(lambda x,y: (x/255,y))

In [ ]:
scaled_train_iterator = train_data.as_numpy_iterator()
scaled_test_iterator = test_data.as_numpy_iterator()
train_batch = scaled_train_iterator.next()
test_batch = scaled_test_iterator.next()
train_batch[0].shape

(32, 256, 256, 3)

In [ ]:
len(train_data)

908

In [ ]:
train_size = int(len(train_data)*0.8)
val_size = int(len(train_data)*0.2)+1
test_size = len(test_data)
train_size+val_size

908

In [ ]:
train = train_data.take(train_size)
val = train_data.skip(train_size).take(val_size)
test = test_data.take(test_size)

In [ ]:
print(len(train),len(val),len(test))

726 182 231


In [ ]:
train_images, train_labels = [], []
val_images, val_labels = [], []


for image, label in train.take(train_size):
    train_images.append(image.numpy())
    train_labels.append(label.numpy())

# Loop through the validation dataset and extract images and labels
for image, label in val.take(val_size):
    val_images.append(image.numpy())
    val_labels.append(label.numpy())

# Convert the lists to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
val_images = np.array(val_images)
val_labels = np.array(val_labels)

KeyboardInterrupt: ignored

In [ ]:
np.save('train_images.npy', train_images)
np.save('train_labels.npy', train_labels)

In [ ]:
train_set = datagen_train.flow_from_directory(
    train_data_dir,
    target_size=(picture_size, picture_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_set = datagen_val.flow_from_directory(
    test_data_dir,
    target_size=(picture_size, picture_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 29031 images belonging to 7 classes.
Found 7378 images belonging to 7 classes.


In [ ]:
data_augmentation = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

In [ ]:
batch_size = 32
train_generator = data_augmentation.flow(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

NameError: ignored

In [ ]:
val_generator = train_data.flow(
    val,
    batch_size=batch_size,
    shuffle=False
)

## Step 3 - Model

In [7]:
model = Sequential(
    [
        Conv2D(64, (3,3), 1,  activation = 'relu', padding='same', input_shape = (picture_size, picture_size, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        BatchNormalization(),

        Conv2D(128, (3,3), 1,  activation = 'relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.25),

        Conv2D(256, (3,3), 1,  activation = 'relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.25),

        Conv2D(128, (3,3), 1,  activation = 'relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.25),

        Conv2D(64, (3,3), 1,  activation = 'relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.25),

        Flatten(),

        Dense(256, activation = 'relu'),
        Dense(7, activation = 'softmax')
    ]
)

In [8]:
model.compile(
    optimizer = Adam(learning_rate = 0.01),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 batch_normalization (BatchN  (None, 24, 24, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 128)      5

In [10]:
log_dir = '/content/gdrive/MyDrive/Human emotions/Logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)

In [11]:
early_stopping = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3, monitor='val_loss', min_lr=0.00001)

In [ ]:
history = model.fit(
    train_set,
    epochs=20,
    validation_data=val_set,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


125/454 [=======>......................] - ETA: 1:29:06 - loss: 1.8580 - accuracy: 0.2501

In [ ]:
model.save('/content/sample_data/Model_Emotion_Detection_in_Live_Video_Data')


In [ ]:

loaded_model = load_model('/content/gdrive/MyDrive/Human emotions/Saved_model')


In [ ]:
predictions = loaded_model.predict(test)

In [ ]:
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
test_labels = np.concatenate([y for x, y in test], axis=0)

In [ ]:
correct_predictions = np.equal(predicted_classes, test_labels)

In [ ]:
accuracy = np.mean(correct_predictions)
print("Test Accuracy:", accuracy)

## Trying to make it work with video frames